In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import plotly.graph_objects as go
import json



# Load your CSV data (update the file path if necessary)
df = pd.read_csv('involvements_history_example_data.csv')

# Parse the pers_involvement_history_json column, which contains JSON-like data
df['involvement_history_parsed'] = df['pers_involvement_history_json'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])






# # CASE WORKER HISTORY
# # Create a new DataFrame to store the parsed involvement history details, including WORKER_ID
# involvement_history_expanded = pd.DataFrame([
#     {'anon_person_id': row['anon_person_id'], 
#      'involvement_id': involvement['INVOLVEMENT_ID'],
#      'involvement_type': involvement['INVOLVEMENT_TYPE_CODE'],
#      'worker_id': involvement['WORKER_ID'],
#      'start_date': pd.to_datetime(involvement['START_DATE']),
#      'end_date': pd.to_datetime(involvement['END_DATE'])}
#     for _, row in df.iterrows() if row['involvement_history_parsed']
#     for involvement in row['involvement_history_parsed']
# ])

# # Filter for only CW involvement type and other conditions
# cw_filtered_data = involvement_history_expanded[
#     (involvement_history_expanded['involvement_type'] == 'CW') &
#     (involvement_history_expanded['start_date'] >= pd.Timestamp('2019-01-01')) &
#     (involvement_history_expanded['end_date'] > involvement_history_expanded['start_date']) &
#     (involvement_history_expanded['end_date'] != pd.Timestamp('1900-01-01'))
# ]

# # Calculate duration in days for each CW involvement
# cw_filtered_data['duration_days'] = (cw_filtered_data['end_date'] - cw_filtered_data['start_date']).dt.days

# # Anonymize the WORKER_IDs with sequential integers
# unique_worker_ids = cw_filtered_data['worker_id'].unique()
# anon_worker_map = {original_id: i+1 for i, original_id in enumerate(unique_worker_ids)}
# cw_filtered_data['anon_worker_id'] = cw_filtered_data['worker_id'].map(anon_worker_map)

# # Select the first 40 individuals with CW involvement for the plot
# individuals_with_data = cw_filtered_data['anon_person_id'].unique()[:40]
# subset_data_filtered = cw_filtered_data[cw_filtered_data['anon_person_id'].isin(individuals_with_data)]

# # Generate unique colors for each anonymized worker ID
# colormap = plt.cm.get_cmap('tab20', len(unique_worker_ids))
# worker_colors = {anon_worker_map[worker_id]: colormap(i) for i, worker_id in enumerate(unique_worker_ids)}

# # Plot the CW involvement history, using duration in days for the x-axis
# plt.figure(figsize=(10, 8))

# # Iterate through each person and plot each CW involvement per anon_worker_id with a unique color
# for i, (anon_id, group) in enumerate(subset_data_filtered.groupby('anon_person_id')):
#     for _, row in group.iterrows():
#         plt.barh(i, row['duration_days'], left=0, color=worker_colors[row['anon_worker_id']], alpha=0.8)

# # Create a legend with a unique entry for each anonymized WORKER_ID
# handles = [plt.Rectangle((0,0),1,1, color=worker_colors[anon_worker_id]) for anon_worker_id in sorted(worker_colors.keys())]
# plt.legend(handles, [f"Worker {anon_worker_id}" for anon_worker_id in sorted(worker_colors.keys())], title="WORKER_IDs", bbox_to_anchor=(1.05, 1), loc='upper left')


# plt.title('Case Worker Involvements Timeline (from 01/01/2019)', fontsize=14)
# plt.xlabel('Duration (Days)', fontsize=12)
# plt.ylabel('Example Person', fontsize=12)
# plt.yticks(ticks=range(len(individuals_with_data)), labels=range(1, len(individuals_with_data) + 1))


# plt.tight_layout()
# plt.grid(True)
# plt.show()



# ### CASEWORKER HISTORY AS SANKEY
# # Create a new DataFrame to store the parsed involvement history details, including WORKER_ID
# involvement_history_expanded = pd.DataFrame([
#     {'anon_person_id': row['anon_person_id'], 
#      'involvement_id': involvement['INVOLVEMENT_ID'],
#      'involvement_type': involvement['INVOLVEMENT_TYPE_CODE'],
#      'worker_id': involvement['WORKER_ID'],
#      'start_date': pd.to_datetime(involvement['START_DATE']),
#      'end_date': pd.to_datetime(involvement['END_DATE'])}
#     for _, row in df.iterrows() if row['involvement_history_parsed']
#     for involvement in row['involvement_history_parsed']
# ])

# # Filter for only CW involvement type and other conditions
# cw_filtered_data = involvement_history_expanded[
#     (involvement_history_expanded['involvement_type'] == 'CW') &
#     (involvement_history_expanded['start_date'] >= pd.Timestamp('2019-01-01')) &
#     (involvement_history_expanded['end_date'] > involvement_history_expanded['start_date']) &
#     (involvement_history_expanded['end_date'] != pd.Timestamp('1900-01-01'))
# ].copy()  # .copy() avoids SettingWithCopyWarning

# # Calculate duration in days for each CW involvement
# cw_filtered_data.loc[:, 'duration_days'] = (cw_filtered_data['end_date'] - cw_filtered_data['start_date']).dt.days

# # Anonymize the WORKER_IDs with sequential integers
# unique_worker_ids = cw_filtered_data['worker_id'].unique()
# anon_worker_map = {original_id: i+1 for i, original_id in enumerate(unique_worker_ids)}
# cw_filtered_data.loc[:, 'anon_worker_id'] = cw_filtered_data['worker_id'].map(anon_worker_map)

# # Select the first 40 individuals with CW involvement for the plot
# individuals_with_data = cw_filtered_data['anon_person_id'].unique()[:40]
# subset_data_filtered = cw_filtered_data[cw_filtered_data['anon_person_id'].isin(individuals_with_data)]

# # Prepare labels for the nodes in the Sankey plot
# person_labels = [f"Person {i+1}" for i in range(len(individuals_with_data))]
# worker_labels = [f"Worker {anon_worker_map[worker_id]}" for worker_id in unique_worker_ids]
# labels = person_labels + worker_labels

# # Create mapping of person IDs and anonymized worker IDs to positions in the labels list
# person_id_map = {anon_person_id: i for i, anon_person_id in enumerate(individuals_with_data)}
# worker_id_map = {anon_worker_map[worker_id]: i + len(individuals_with_data) for i, worker_id in enumerate(unique_worker_ids)}


# # Prepare source, target, and value lists for the Sankey plot
# sources = []
# targets = []
# values = []

# for _, row in subset_data_filtered.iterrows():
#     sources.append(person_id_map[row['anon_person_id']])
#     targets.append(worker_id_map[row['anon_worker_id']])
#     values.append(row['duration_days'])  # Use duration as the value (thickness of the flow)

# # Create the Sankey chart
# fig = go.Figure(go.Sankey(
#     node=dict(
#         pad=15,
#         thickness=20,
#         line=dict(color="black", width=0.5),
#         label=labels
#     ),
#     link=dict(
#         source=sources,  # indices of sources (persons)
#         target=targets,  # indices of targets (anonymized workers)
#         value=values     # durations in days
#     )
# ))

# # Customize the layout
# fig.update_layout(title_text="CW Involvements Timeline by Anonymized WORKER_ID (Duration as Flow Thickness)", font_size=10)

# # Show the plot
# fig.show()







# WITHOUT CODE_DESC ADDED
# Create a new DataFrame to store the parsed involvement history details
involvement_history_expanded = pd.DataFrame([
    {'anon_person_id': row['anon_person_id'], 
     'involvement_id': involvement['INVOLVEMENT_ID'],
     'involvement_type': involvement['INVOLVEMENT_TYPE_CODE'],
     'start_date': pd.to_datetime(involvement['START_DATE']),
     'end_date': pd.to_datetime(involvement['END_DATE'])}
    for _, row in df.iterrows() if row['involvement_history_parsed']
    for involvement in row['involvement_history_parsed']
])

# Apply the filter conditions
involvement_history_expanded_filtered = involvement_history_expanded[
    (involvement_history_expanded['start_date'] >= pd.Timestamp('2019-01-01')) &
    (involvement_history_expanded['end_date'] > involvement_history_expanded['start_date']) &
    (involvement_history_expanded['end_date'] != pd.Timestamp('1900-01-01'))
]

# Calculate duration in days for each involvement
involvement_history_expanded_filtered['duration_days'] = (involvement_history_expanded_filtered['end_date'] - involvement_history_expanded_filtered['start_date']).dt.days

# Get distinct involvement types and rank them by frequency of occurrence
top_30_involvement_types = involvement_history_expanded_filtered['involvement_type'].value_counts().index[:40]

# Filter the data to include only the selected top 30 involvement types
subset_data_filtered = involvement_history_expanded_filtered[involvement_history_expanded_filtered['involvement_type'].isin(top_30_involvement_types)]

# Select only individuals who have data for the top 30 involvement types, ensuring there are no empty traces
individuals_with_data = subset_data_filtered['anon_person_id'].unique()[:40]
subset_data_filtered = subset_data_filtered[subset_data_filtered['anon_person_id'].isin(individuals_with_data)]

# Use a colormap for 30 unique involvement types
colormap = plt.cm.get_cmap('tab20', 30)  # 'tab20' can handle up to 20, so we'll repeat the scheme for 30 types

# Map involvement types to colors
involvement_colors = {inv_type: colormap(i % 20) for i, inv_type in enumerate(top_30_involvement_types)}

# Plot the involvement history, using duration in days for the x-axis
plt.figure(figsize=(10, 8))

for i, (anon_id, group) in enumerate(subset_data_filtered.groupby('anon_person_id')):
    for _, row in group.iterrows():
        plt.barh(i, row['duration_days'], left=0, color=involvement_colors[row['involvement_type']], alpha=0.8)

# Add a legend for the top 30 involvement types
handles = [plt.Rectangle((0,0),1,1, color=involvement_colors[inv_type]) for inv_type in top_30_involvement_types]
plt.legend(handles, top_30_involvement_types, title="Involvement Types", bbox_to_anchor=(1.05, 1), loc='upper left')

# Customize the plot
plt.title('Involvements Timeline (from 01/01/2019)', fontsize=14)
plt.xlabel('Duration (Days)', fontsize=12)
plt.ylabel('Example Person', fontsize=12)
plt.yticks(ticks=range(len(individuals_with_data)), labels=range(1, len(individuals_with_data) + 1))

# Show the plot
plt.tight_layout()
plt.grid(True)
plt.show()






## SANKEY DIAGRAM

# # Create a new DataFrame to store the parsed involvement history details, including start and end dates
# involvement_history_expanded = pd.DataFrame([
#     {'anon_person_id': row['anon_person_id'], 
#      'involvement_type': involvement['INVOLVEMENT_TYPE_CODE'],
#      'start_date': pd.to_datetime(involvement['START_DATE']),
#      'end_date': pd.to_datetime(involvement['END_DATE'])}
#     for _, row in df.iterrows() if row['involvement_history_parsed']
#     for involvement in row['involvement_history_parsed']
# ])

# # Apply the filter for valid data (post-2019) and remove invalid end dates
# involvement_history_expanded_filtered = involvement_history_expanded[
#     (involvement_history_expanded['start_date'] >= pd.Timestamp('2019-01-01')) &
#     (involvement_history_expanded['end_date'] > involvement_history_expanded['start_date']) &
#     (involvement_history_expanded['end_date'] != pd.Timestamp('1900-01-01'))
# ]

# # Calculate the duration in days
# involvement_history_expanded_filtered['duration_days'] = (involvement_history_expanded_filtered['end_date'] - involvement_history_expanded_filtered['start_date']).dt.days

# # Filter by the top 30 involvement types based on frequency
# top_30_involvement_types = involvement_history_expanded_filtered['involvement_type'].value_counts().index[:30]
# subset_data_filtered = involvement_history_expanded_filtered[involvement_history_expanded_filtered['involvement_type'].isin(top_30_involvement_types)]

# # Get unique person IDs and limit to the first 10 people (or another number you prefer)
# unique_persons = subset_data_filtered['anon_person_id'].unique()[:3]
# unique_involvement_types = top_30_involvement_types

# # Create the node labels (person IDs and involvement types)
# labels = [f'Person {i}' for i in range(1, len(unique_persons) + 1)] + list(unique_involvement_types)

# # Create a mapping for person IDs and involvement types to their indices in the labels list
# person_id_map = {person: i for i, person in enumerate(unique_persons)}
# involvement_type_map = {inv_type: i + len(unique_persons) for i, inv_type in enumerate(unique_involvement_types)}

# # Create source and target indices for the Sankey chart based on durations
# sources = []
# targets = []
# values = []

# # Iterate over the data to create the links between persons and involvement types based on duration
# for _, row in subset_data_filtered.iterrows():
#     if row['anon_person_id'] in person_id_map:  # Only include persons within the top 10
#         sources.append(person_id_map[row['anon_person_id']])
#         targets.append(involvement_type_map[row['involvement_type']])
#         values.append(row['duration_days'])  # Use duration in days as the value

# # Create the Sankey chart
# fig = go.Figure(go.Sankey(
#     node=dict(
#         pad=15,
#         thickness=20,
#         line=dict(color="black", width=0.5),
#         label=labels
#     ),
#     link=dict(
#         source=sources,  # indices of sources (persons)
#         target=targets,  # indices of targets (involvement types)
#         value=values     # durations in days
#     )
# ))

# # Customize the layout
# fig.update_layout(title_text=" Sankey: Involvement Types per Person/Duration", font_size=10)

# # Show the plot
# fig.show()



TypeError: string indices must be integers, not 'str'